# 정수 인코딩
- 텍스트를 숫자로 변경한다.
- 단어의 등장 빈도수를 기준으로 정렬하고 부여한다.

In [4]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

raw_text = "A barber is a person. a barber is good person. a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain."

sentences = sent_tokenize(raw_text)
print(sentences)


['A barber is a person.', 'a barber is good person.', 'a barber is huge person.', 'he Knew A Secret!', 'The Secret He Kept is huge secret.', 'Huge secret.', 'His barber kept his word.', 'a barber kept his word.', 'His barber kept his secret.', 'But keeping and keeping such a huge secret to himself was driving the barber crazy.', 'the barber went up a huge mountain.']


문장단위로 토큰화를 진행 완료

- 정제 작업과 정규화 작업을 병행하여 단어 토큰화 수행
- 단어들을 소문자화해서 단어의 개수를 통일, 불용어와 단어 길이가 2이하인 경우에 일부 제외
- 단어가 텍스트일 때만 할 수 있는 전처리는 끝내놓아야 한다.

In [ ]:
vocab = {}
preprocessed_sentences = []
stop_words = set(stopwords.words('english'))

for sentence in sentences:
  # 단어 토큰화
  tokenized_sentence = word_tokenize(sentence)
  
  result = []
  for word in tokenized_sentence:
    # 소문자화
    word = word.lower()
    if word not in stop_words:
      # 단어 길이가 2이상인 경우에만 추가
      if len(word) > 2:
        result.append(word)
        # 단어가 처음 등장하는 경우 0으로 초기화
        if word not in vocab:
          vocab[word] = 0
        # 단어가 등장할 때마다 1씩 증가
        vocab[word] += 1
  preprocessed_sentences.append(result)
print(preprocessed_sentences)
print(vocab)



        


[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]
{'barber': 8, 'person': 3, 'good': 1, 'huge': 5, 'knew': 1, 'secret': 6, 'kept': 4, 'word': 2, 'keeping': 2, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1}


In [8]:
vocab_sorted = sorted(vocab.items(), key = lambda x:x[1], reverse = True)
print(vocab_sorted)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3), ('word', 2), ('keeping', 2), ('good', 1), ('knew', 1), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)]


높은 빈도를 가질 수록 낮은 정수를 부여, 1부터 시작
- 빈도수가 적은 단어는 자연어 처리에서 의미를 가지지 않을 가능성이 높다

In [9]:
word_to_index = {}
i = 0
for (word, frequency) in vocab_sorted :
    if frequency > 1 : # 빈도수가 작은 단어는 제외.
        i = i + 1
        word_to_index[word] = i

print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7}


In [ ]:
vocab_size = 5

# 인덱스가 5 초과인 단어 제거
words_frequency = [word for word, index in word_to_index.items() if index >= vocab_size + 1]

for w in words_frequency:
  del word_to_index[w]

print(word_to_index)




{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


사전에 없는 단어는 Out-Of-Vocabulary 문제라고 한다.
없는 단어는 OOV로 인코딩한다.

In [11]:
word_to_index['OOV'] = len(word_to_index) + 1

print(word_to_index)





{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'OOV': 6}


In [12]:
encoded_sentences = []
for sentence in preprocessed_sentences:
    encoded_sentence = []
    for word in sentence:
        try:
            # 단어 집합에 있는 단어라면 해당 단어의 정수를 리턴.
            encoded_sentence.append(word_to_index[word])
        except KeyError:
            # 만약 단어 집합에 없는 단어라면 'OOV'의 정수를 리턴.
            encoded_sentence.append(word_to_index['OOV'])
    encoded_sentences.append(encoded_sentence)
print(encoded_sentences)


[[1, 5], [1, 6, 5], [1, 3, 5], [6, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [6, 6, 3, 2, 6, 1, 6], [1, 6, 3, 6]]


## Counter 사용하기

In [17]:
from collections import Counter

print(preprocessed_sentences)

all_words_list = sum(preprocessed_sentences, [])
print(all_words_list)

vocab = Counter(all_words_list)
print(vocab)

vocab_size = 5 # 등장빈도 상위 5개만 보기
vocab = vocab.most_common(vocab_size)
print(vocab)

word_to_index = {}
i = 0
for (word, frequency) in vocab:
  if frequency > 1:
    i = i + 1
    word_to_index[word] = i

print(word_to_index)











[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]
['barber', 'person', 'barber', 'good', 'person', 'barber', 'huge', 'person', 'knew', 'secret', 'secret', 'kept', 'huge', 'secret', 'huge', 'secret', 'barber', 'kept', 'word', 'barber', 'kept', 'word', 'barber', 'kept', 'secret', 'keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy', 'barber', 'went', 'huge', 'mountain']
Counter({'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3, 'word': 2, 'keeping': 2, 'good': 1, 'knew': 1, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1})
[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]
{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


## NLTK의 FreqDist 사용하기

In [ ]:
from nltk import FreqDist
import numpy as np

vocab = FreqDist(np.hstack(preprocessed_sentences))
print(vocab['barber'])

vocab_size = 5
vocab = vocab.most_common(vocab_size)
print(vocab)

word_to_index = { word[0]:index+1 for index, word in enumerate(vocab) } # enumerate는 인덱스와 값을 동시에 반환
print(word_to_index)


8
[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]
{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


# Keras의 텍스트 전처리

In [38]:
from tensorflow.keras.preprocessing.text import Tokenizer

print(preprocessed_sentences)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_sentences)

print(tokenizer.word_index)
print(tokenizer.word_counts)
print(tokenizer.texts_to_sequences(preprocessed_sentences))

vocab_size = 5
tokenizer = Tokenizer(num_words = vocab_size + 1)
tokenizer.fit_on_texts(preprocessed_sentences)

print(tokenizer.texts_to_sequences(preprocessed_sentences))





[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]
{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}
OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])
[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]
[[1, 5], [1, 5], [1, 3, 5], [2], [2, 4, 3, 2], [3, 2], [1, 4], [1, 4], [1, 4, 2], [3, 2, 1], [1, 3]]


케라스 토크나이저는 OOV에 대해서는 정수로 바꾸는 과정에서 단어를 제거한다.

OOV로 간주해서 보존하고 싶다면 oov_token을 사용

In [41]:
vocab_size = 5
tokenizer = Tokenizer(num_words = vocab_size + 2, oov_token = 'OOV')
tokenizer.fit_on_texts(preprocessed_sentences)

print(tokenizer.texts_to_sequences(preprocessed_sentences))

[[2, 6], [2, 1, 6], [2, 4, 6], [1, 3], [3, 5, 4, 3], [4, 3], [2, 5, 1], [2, 5, 1], [2, 5, 3], [1, 1, 4, 3, 1, 2, 1], [2, 1, 4, 1]]
